In [1]:
import extract
import pandas as pd
import os
import geopandas as gpd

In [2]:
import importlib
importlib.reload(extract)

<module 'extract' from '/Users/kguan17/Documents/GitHub/RA-Work/acs/extract.py'>

In [3]:
# open state_fips
state_fips = pd.read_csv('state_fips.csv')

In [10]:
# for every folder in raw_tract_data
for folder in os.listdir('raw_tract_data'):
    # split folder into list by underscore
    folder_tokens = folder.split('_')
    
    # get state column from state_fips where code is third item in list as int
    state = state_fips[state_fips['code'] == int(folder_tokens[2])]['state'].values[0].lower()

    # if last item in folder is 'tract' year is 2000, otherwise it is 20 + last two digits
    year = '2020' if folder_tokens[-1] == 'tract' else '20' + folder_tokens[-1][-2:]

    # if tract file already exists, skip
    if os.path.exists(f'tract_polygons/{state}_tracts_{year}.csv'):
        continue

    # get the filepath of the shp file in the folder with the same name
    shp_file = os.path.join('raw_tract_data', folder, folder + '.shp')

    tract_geos = extract.read_tracts(shp_file)
    
    # save as acs\tract_polygons\berkeley_tracts_2010.csv
    tract_geos.to_csv(f'tract_polygons/{state}_tracts_{year}.csv', index=False)

In [46]:
# for alaska upper (weird KML)
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
districts = gpd.read_file('raw_geo_data\\alaska_upper_2022.kml', driver='KML')
# rename Name to district
districts = districts.rename(columns={'Name': 'district'})
# only keep district and geometry
districts = districts[['district', 'geometry']]
# only keep the part of the district field in the form 0200A
districts['district'] = districts['district'].str.extract(r'(\d{4}[A-Z])')
# only keep the last letter
districts['district'] = districts['district'].str[-1]
# save to geo_polygons folder as alaska_upper_districts_2022.csv
# districts.to_csv('geo_polygons/alaska_upper_districts_2022.csv', index=False)

In [10]:
# for SHP
# read in shp
districts = gpd.read_file(r'raw_geo_data/Corvallis Boundary/Ward.shp')
districts

,WARD,HTML,COUNCIL_NA,COUNCIL_PH,ADDR1,ZIP,HOME_PHONE,EMAIL,LINK,AREA_SQMI,ACRES,X_COORD,Y_COORD,DATE_PUBLI,Shape_Leng,Shape_Area,geometry
0,1,https://archives.corvallisoregon.gov/internal/...,Jan Napack,541-766-6491,4998 SW Hollyhock Circle,97333,None,Jan.napack@corvallisoregon.gov,https://www.corvallisoregon.gov/directory-list...,2.440,1773.334998,7.471267e+06,334710.86855,2023-01-03,80465.626455,7.724647e+07,"POLYGON ((7476893.776 333908.477, 7476883.381 ..."
1,2,https://archives.corvallisoregon.gov/internal/...,Briae Lewis,541-766-6492,124 NW 7th Street,97330,None,Briae.Lewis@corvallisoregon.gov,https://www.corvallisoregon.gov/directory-list...,1.033,824.339186,7.481542e+06,337744.97297,2023-01-03,53135.996826,3.590821e+07,"POLYGON ((7478713.450 341465.276, 7478718.074 ..."
2,3,https://archives.corvallisoregon.gov/internal/...,Hyatt Lytle,541-766-6493,"2766 SW Pickford Street, #94",97333,None,Hyatt.lytle@corvallisoregon.gov,https://www.corvallisoregon.gov/directory-list...,2.018,1351.000360,7.481260e+06,329065.80022,2023-01-03,69432.683681,5.884958e+07,"POLYGON ((7481292.734 333946.699, 7481305.385 ..."
3,4,https://archives.corvallisoregon.gov/internal/...,Gabe Shepherd,541-766-6494,1445 SW A Ave APT 17,97330,None,Gabe.shepherd@corvallisoregon.gov,https://www.corvallisoregon.gov/directory-list...,1.408,424.041495,7.476683e+06,338815.15428,2023-01-03,22468.836521,1.847125e+07,"POLYGON ((7476485.389 338927.277, 7476450.535 ..."
4,5,https://archives.corvallisoregon.gov/internal/...,Charlyn Ellis,541-766-6495,519 NW 21st Street,97330,None,Charlyn.ellis@corvallisoregon.gov,https://www.corvallisoregon.gov/directory-list...,0.643,427.877260,7.479730e+06,343465.13443,2023-01-03,20688.704427,1.863833e+07,"POLYGON ((7479711.273 341971.592, 7479654.987 ..."
5,6,https://archives.corvallisoregon.gov/internal/...,Laurie Chaplen,541-766-6496,1250 NW Garfield Ave,97330,None,Laurie.chaplen@corvallisoregon.gov,https://www.corvallisoregon.gov/directory-list...,2.149,1392.895755,7.483283e+06,346693.09146,2023-01-03,47267.229039,6.067454e+07,"POLYGON ((7482490.574 343694.383, 7482705.239 ..."
6,7,https://archives.corvallisoregon.gov/internal/...,Paul Shaffer,541-766-6497,3338 NW McKinley Drive,97330,None,Paul.shaffer@corvallisoregon.gov,https://www.corvallisoregon.gov/directory-list...,1.312,909.118739,7.474905e+06,345915.25631,2023-01-03,45554.210388,3.960121e+07,"POLYGON ((7477663.340 347067.146, 7477960.159 ..."
7,8,https://archives.corvallisoregon.gov/internal/...,Tracey Yee,541-766-6498,2609 NW Fireweed Pl,97330,None,Tracey.Yee@corvallisoregon.gov,https://www.corvallisoregon.gov/directory-list...,2.145,1471.795467,7.474874e+06,351438.60698,2023-01-03,52834.972101,6.411141e+07,"POLYGON ((7472012.365 352808.525, 7472003.439 ..."
8,9,https://archives.corvallisoregon.gov/internal/...,Tony Cadena,541-766-6499,548 NW Mirador Pl,97330,None,Tony.Cadena@corvallisoregon.gov,https://www.corvallisoregon.gov/directory-list...,1.231,801.316567,7.486241e+06,351609.31747,2023-01-03,32468.698892,3.490535e+07,"POLYGON ((7487548.667 349406.454, 7486998.277 ..."


In [11]:
# only keep district and geometry
district_name = 'WARD'
year=2022
desc='corvallis_city_council'
districts = districts[[district_name, 'geometry']]
# rename to district and convert to int
districts = districts.rename(columns={district_name: 'district'})
districts['district'] = districts['district'].astype(int)

In [12]:
districts

,district,geometry
0,1,"POLYGON ((7476893.776 333908.477, 7476883.381 ..."
1,2,"POLYGON ((7478713.450 341465.276, 7478718.074 ..."
2,3,"POLYGON ((7481292.734 333946.699, 7481305.385 ..."
3,4,"POLYGON ((7476485.389 338927.277, 7476450.535 ..."
4,5,"POLYGON ((7479711.273 341971.592, 7479654.987 ..."
5,6,"POLYGON ((7482490.574 343694.383, 7482705.239 ..."
6,7,"POLYGON ((7477663.340 347067.146, 7477960.159 ..."
7,8,"POLYGON ((7472012.365 352808.525, 7472003.439 ..."
8,9,"POLYGON ((7487548.667 349406.454, 7486998.277 ..."


In [13]:
# save to geo_polygons folder
districts.to_csv(f'geo_polygons/{desc}_{year}.csv', index=False)